测试：

1、使用 litellm 框架调用大模型

2、使用 openai-agent 框架搭建 agent

## 基于 litellm 调用大模型

使用 litellm 库调用模型，支持的[大模型提供商](https://docs.litellm.ai/docs/providers)

In [ ]:
import os

from dotenv import load_dotenv
from litellm import completion

load_dotenv()

# 基于 litellm，测试三类大模型：deepseek、glm4、mastral

# api_key = os.getenv('DEEPSEEK_API_KEY')
# base_url = "https://api.deepseek.com/v1"
# chat_model = "deepseek/deepseek-chat"

chat_model = "openai/glm-4-flash-250414"  # openai-compatible
base_url="https://open.bigmodel.cn/api/paas/v4"
api_key=os.getenv('zhipu_key')

# chat_model = "mistral/mistral-small-latest"
# base_url="https://api.mistral.ai/v1"
# api_key=os.getenv('mistral_key')

response = completion(
    model=chat_model,
    messages=[{"content": "你有什么技能？", "role": "user"}],
    api_base=base_url,
    api_key=api_key
)
print(response.choices[0].message.content)

## 基于 openai-agent 搭建 agent 框架

### 方法一

In [ ]:
from agents import Agent, Runner, set_tracing_disabled, set_default_openai_api, set_default_openai_client
from openai import AsyncOpenAI
import os
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()

# 如果切换模型，更改下面的参数即可
chat_model = "deepseek-chat"
client = AsyncOpenAI(
    base_url="https://api.deepseek.com/v1",
    api_key=os.getenv('DEEPSEEK_API_KEY'),
)

# chat_model = "glm-4-flash-250414"
# client = AsyncOpenAI(
#     base_url="https://open.bigmodel.cn/api/paas/v4/",
#     api_key=os.getenv('zhipu_key'),
# )

# chat_model = "mistral-small-latest"
# # 支持并发处理
# client = AsyncOpenAI(
#     base_url="https://api.mistral.ai/v1",
#     api_key=os.getenv('mistral_key'),
# )

# agent 框架默认使用刚配置的 client
# 禁止 client 的追踪功能
set_default_openai_client(client=client, use_for_tracing=False)
# 使用 chat_completion 接口
set_default_openai_api("chat_completions")
# 全局禁用追踪
set_tracing_disabled(disabled=True)

agent = Agent(name="Assistant", model=chat_model, instructions="You are a helpful assistant")
# run_sync() 同步运行，阻塞当前线程直到 agent 执行完成
# jupyter notebook 为异步环境（同时运行多个 notebook），所以运行下面代码会报错
# result = Runner.run_sync(agent, "给我讲个程序员相亲的笑话")

# Runner.run 是异步方法，返回一个协程对象，await 会等待协程执行完成并返回结果
result = await Runner.run(agent, "给我讲个程序员相亲的笑话")
print(result.final_output)

### 方法二

In [ ]:
# 下面代码应该在 py 文件中运行
from agents import Agent, ModelSettings, enable_verbose_stdout_logging, Runner
from agents.extensions.models.litellm_model import LitellmModel
import os
import asyncio
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()

# 从环境变量中读取api_key
chat_model = "openai/glm-4-flash-250414"  # openai-compatible
base_url="https://open.bigmodel.cn/api/paas/v4"
api_key=os.getenv('zhipu_key')

# 调用函数，启用详细日志输出。默认输出警告和错误信息。可以使用 logging 库记录更加详细的日志信息
# enable_verbose_stdout_logging()
set_tracing_disabled(disabled=True)

llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)

agent = Agent(name="Assistant", 
              model=llm, 
              instructions="You are a helpful assistant",
              model_settings=ModelSettings(temperature=0.1)  # 配置温度等参数
              )

async def main():
    result = await Runner.run(agent, "给我讲个程序员相亲的笑话")
    print(result.final_output)

# if __name__ == "__main__":
#     asyncio.run(main())

await main()

为什么存在这么多无关输出？

```
Tracing is disabled. Not creating trace Agent workflow
Setting current trace: no-op
Tracing is disabled. Not creating span <agents.tracing.span_data.AgentSpanData object at 0x748cbe9c1760>
Running agent Assistant (turn 1)
Tracing is disabled. Not creating span <agents.tracing.span_data.GenerationSpanData object at 0x748cb62029e0>
Calling Litellm model: openai/glm-4-flash-250414
[
  {
    "content": "You are a helpful assistant",
    "role": "system"
  },
  {
    "role": "user",
    "content": "给我讲个程序员相亲的笑话"
  }
]
Tools:
[]
Stream: False
Tool choice: NOT_GIVEN
Response format: NOT_GIVEN

LLM resp:
{
  "content":
```

## 异步运行

智谱、deepseek、和mistral都不支持异步运行，需要用Runner.run_sync 去运行

可以借助 openai 官方提供的 LitellmModel 配合 mistral，支持异步运行

❓不理解

In [ ]:
from agents import Agent, Runner, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
import os
import asyncio
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
# 从环境变量中读取api_key
api_key = os.getenv('mistral_key')
base_url = 'https://api.mistral.ai/v1'
chat_model = "mistral/mistral-small-latest"

set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)
agent = Agent(name="Assistant", model=llm, instructions="You are a helpful assistant")


# 下面的代码可以运行，但是存在大量无关输出！
async def main():
    result = await Runner.run(agent, "给我讲个程序员相亲的笑话")
    print(result.final_output)

# 在 Notebook 中直接 await
await main()

好的，这里有一个关于程序员相亲的笑话，希望能让你会心一笑：

---

有一个程序员去相亲，见面后女生问他：“你平时都喜欢做什么呀？”

程序员想了想，回答：“我喜欢写代码，解决问题，还有……调试。”

女生有点困惑：“调试？什么意思？”

程序员解释道：“就是当代码出现问题时，我会花很多时间找出错误，然后修复它。”

女生点点头：“听起来很有趣。那你平时都写什么代码呢？”

程序员自豪地说：“我最近在写一个相亲APP，专门帮助像我这样的程序员找到合适的伴侣。”

女生惊讶地问：“真的吗？那你的APP有什么特别之处？”

程序员得意地回答：“它会根据用户的代码风格、调试习惯和对技术的热情来匹配合适的伴侣。”

女生笑了笑：“那你的APP有没有帮你找到合适的伴侣呢？”

程序员尴尬地挠了挠头：“还在调试中……不过，我今天来相亲，就是为了收集更多的用户反馈。”

---

希望这个笑话能让你开心一笑！
